In [1]:
# To install the grein_loader package
#!pip install grein_loader
!pwd

/home/zplumridge/biobombe/0.expression-download


In [2]:
import grein_loader as loader
import pandas as pd
import os
import time

In [3]:
# Load log file from disk to see if this data has already been added
def load_log(logFile):
    log = pd.read_csv(logFile, names=["GSE", "status"])
    log = log[log["status"] == "end"]
    return set(log["GSE"].unique())

In [4]:
def load_data(geo_accession, count_matrix_df):
    description, metadata, count_matrix = loader.load_dataset(geo_accession)
        
    # Merge in to existing count matrix DF
    if count_matrix_df.empty:
        return count_matrix
    else:
        return pd.merge(count_matrix_df, count_matrix,  how='left', on=["gene", "gene_symbol"], suffixes=(None, "_y"))

In [5]:
# Loading grein overview

# loading a subset of the data
# number_of_datasets = 1000
# overview = loader.load_overview(number_of_datasets)

# loading all the data
overview = loader.load_overview()

# Accessing the geo_accession ID and study species from overview
geo_accession_ids = []
species = []
for i in range(len(overview)):
    geo_accession_ids.append(overview[i]['geo_accession'])
    species.append(overview[i]['species'])

In [7]:
# saving the species in each GSE
GSE_species = {'geo_accession_id': geo_accession_ids, 'species': species}
GSE_species_df = pd.DataFrame.from_dict(GSE_species)

# Downloading Human Data

In [18]:
# Prepare to download human data

# Filter dataframe for human data only
GSE_species_df_filtered = GSE_species_df[GSE_species_df['species'] == 'Homo sapiens']

GSE_species_df_filtered

# Check if there is already expression data downloaded or whether we are starting fresh
humanLog = "greinLoadHuman.log"
humanData = "download/grein_count_matrix_human.pkl"
if os.path.exists(humanLog) and os.path.exists(humanData): 
    print("...loading human data")
    count_matrix_human_df = pd.read_pickle(humanData)
    completed = load_log(humanLog)
else:
    print("starting human download from scratch")
    count_matrix_human_df = pd.DataFrame()
    completed = set()
    !touch {humanLog}

...loading human data


In [ ]:
# Saving human gene expression data
with open(humanLog, "a") as logfile:
    for geo_accession in GSE_species_df_filtered['geo_accession_id']:
        
        # Check if this data was logged as loaded
        if geo_accession in completed:
            print("Already done: {0}".format(geo_accession))
            continue
            
        # If new data, download data and add to df
        print("********* |", geo_accession, "| ***********")
        logfile.write(geo_accession + ",start\n")
        
        count_matrix_human_df = load_data(geo_accession, count_matrix_human_df)
        
        logfile.write(geo_accession + ",write\n")
        count_matrix_human_df.to_pickle(humanData)
        logfile.write(geo_accession + ",end\n")
        


In [18]:
pd.read_pickle(humanData)

,gene,gene_symbol,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,...,GSM2692393,GSM2692394,GSM2692395,GSM2692396,GSM2692397,GSM2692398,GSM2692399,GSM2692400,GSM2692401,GSM2692402
0,ENSG00000000003,TSPAN6,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,...,36.5392,18.2983,28.3281,8.5205,17.8816,20.3450,17.4323,11.6830,11.4493,6.7943
1,ENSG00000000005,TNMD,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,...,0.0000,0.0000,2.9989,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,ENSG00000000419,DPM1,1097.7855,367.2333,316.5226,3895.0588,1536.1375,1084.5276,127.7205,245.1014,...,152.1948,106.2851,223.5236,171.2681,188.3446,133.3678,127.3812,223.1960,165.0938,108.3981
3,ENSG00000000457,SCYL3,601.4702,270.9239,163.6869,1177.2241,873.9074,845.1423,229.8548,134.4694,...,378.9609,252.2508,307.5954,264.8942,297.0646,373.8453,272.0667,166.7782,220.1159,147.8496
4,ENSG00000000460,C1orf112,1040.8861,349.1418,298.1800,1967.3975,2115.2425,980.9359,253.4457,145.7038,...,141.2696,121.5965,555.7400,364.5267,360.3099,158.3518,89.2894,73.9050,56.6242,51.1808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27985,ENSG00000283688,MIR6715B,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
27986,ENSG00000283690,MIR3116-2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
27987,ENSG00000283694,MIR3202-2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
27988,ENSG00000283697,LOC101928917,1.7315,0.0000,0.0000,0.0000,2.0039,3.9899,2.5184,4.1298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Downloading Rat Data
(only for testing with a comparatively small dataset)

In [6]:
# Prepare to download mouse data

# Filter dataframe for human data only
GSE_species_df_filtered = GSE_species_df[GSE_species_df['species'] == 'Rattus norvegicus']

GSE_species_df_filtered

# Check if there is already expression data downloaded or whether we are starting fresh
ratLog = "greinLoadRat.log"
ratData = "download/grein_count_matrix_rat.pkl"
if os.path.exists(ratLog) and os.path.exists(ratData): 
    print("...loading rat data")
    count_matrix_rat_df = pd.read_pickle(ratData)
    completed = load_log(ratLog)
else:
    print("starting rat download from scratch")
    count_matrix_rat_df = pd.DataFrame()
    completed = set()
    !touch {ratLog}

starting human download from scratch


In [7]:
# Saving rat gene expression data
with open(ratLog, "a") as logfile:
    for geo_accession in GSE_species_df_filtered['geo_accession_id']:
        
        # Check if this data was logged as loaded
        if geo_accession in completed:
            print("Already done: {0}".format(geo_accession))
            continue
            
        # If new data, download data and add to df
        print("********* |", geo_accession, "| ***********")
        logfile.write(geo_accession + ",start\n")
        
        count_matrix_rat_df = load_data(geo_accession, count_matrix_rat_df)
        
        logfile.write(geo_accession + ",write\n")
        count_matrix_rat_df.to_pickle(ratData)
        logfile.write(geo_accession + ",end\n")

********* | GSE100021 | ***********
********* | GSE100203 | ***********
********* | GSE100454 | ***********
********* | GSE101153 | ***********
********* | GSE101798 | ***********


KeyboardInterrupt: 

In [8]:
pd.read_pickle(ratData)

,gene,gene_symbol,GSM2668015,GSM2668016,GSM2668017,GSM2668018,GSM2668019,GSM2668020,GSM2668021,GSM2668022,...,GSM2683309,GSM2683310,GSM2683311,GSM2683312,GSM2699989,GSM2699990,GSM2699991,GSM2699992,GSM2699993,GSM2699994
0,ENSRNOG00000000007,Gad1,5140.2551,2873.5173,3494.3567,2488.5614,2180.0086,4054.4603,3249.2422,2984.1460,...,55.3001,60.6636,64.8627,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,ENSRNOG00000000008,Alx4,88.3117,54.3955,68.8133,54.5058,61.3839,84.8890,92.7423,62.1657,...,67.0283,61.0076,92.9832,75.9176,10.0072,10.2904,12.9965,5.2917,2.9983,2.9982
2,ENSRNOG00000000009,Tmco5b,71.9091,47.7813,50.4332,41.1147,40.6470,56.4535,48.4759,47.5353,...,83.0249,56.5682,75.6148,44.3473,0.0000,0.0000,1.9925,0.0000,0.0000,0.0000
3,ENSRNOG00000000010,Cbln1,366.8392,236.4766,388.1470,212.8647,218.2449,392.4068,325.6375,231.4440,...,141.5510,100.1295,143.5802,87.1007,80.7889,95.8700,88.7592,0.0000,0.9978,0.0000
4,ENSRNOG00000000012,Tcf15,164.0565,86.1982,86.2113,86.0566,100.7740,145.9362,98.5538,97.6332,...,47.3560,26.0843,30.6765,43.2454,9.9237,13.7869,6.8718,3.3902,5.8108,10.7463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18166,ENSRNOG00000051309,Olr1202,50.0676,46.8461,45.3128,25.3502,26.2111,55.1960,33.0198,23.0041,...,48.1882,24.1112,32.8681,28.5335,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
18167,ENSRNOG00000051323,Olr1213,8.2000,12.6561,10.9044,9.6830,2.0186,13.7930,16.2670,9.7385,...,22.1623,16.7260,21.8294,13.5423,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
18168,ENSRNOG00000051329,LOC103691740,19.3633,15.2961,13.0000,19.0785,5.0209,7.0169,10.0205,16.0799,...,44.9918,26.0032,44.9920,14.0013,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
18169,ENSRNOG00000051344,Psme3,3272.6305,2132.6875,2453.4837,2007.0588,1866.3344,2881.3519,2529.0462,2058.9222,...,4189.2189,2618.3175,4523.2008,4095.3672,799.0016,1318.5732,1189.8005,2421.2753,2276.9981,2071.7715


# Downloading Mouse Data

In [11]:
# Prepare to download mouse data

# Filter dataframe for mouse data only
GSE_species_df_filtered = GSE_species_df[GSE_species_df['species'] == 'Mus musculus']

GSE_species_df_filtered

# Check if there is already expression data downloaded or whether we are starting fresh
miceLog = "greinLoadMice.log"
miceData = "download/grein_count_matrix_mice.pkl"
if os.path.exists(miceLog) and os.path.exists(miceData): 
    print("...loading mouse data")
    count_matrix_mice_df = pd.read_pickle(miceData)
    completed = load_log(miceLog)
else:
    print("starting mouse download from scratch")
    count_matrix_mice_df = pd.DataFrame()
    completed = set()
    !touch {miceLog}

starting mouse download from scratch


In [12]:
# Saving mouse gene expression data
with open(miceLog, "a") as logfile:
    for geo_accession in GSE_species_df_filtered['geo_accession_id']:
        
        # Check if this data was logged as loaded
        if geo_accession in completed:
            print("Already done: {0}".format(geo_accession))
            continue
            
        # If new data, download data and add to df
        print("********* |", geo_accession, "| ***********")
        logfile.write(geo_accession + ",start\n")
        
        count_matrix_mice_df = load_data(geo_accession, count_matrix_mice_df)
        
        logfile.write(geo_accession + ",write\n")
        count_matrix_mice_df.to_pickle(miceData)
        logfile.write(geo_accession + ",end\n")

********* | GSE100035 | ***********
********* | GSE100039 | ***********
********* | GSE100067 | ***********
********* | GSE100070 | ***********
********* | GSE100098 | ***********


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [13]:
pd.read_pickle(miceLog)


UnpicklingError: invalid load key, ','.

In [ ]:
# Loading data from the data sets using the geo_accession IDs
# NOTE this does not save/pickle the description or metadata at all

with open("greinLoad.log", "a") as logfile:
    for geo_accession in geo_accession_ids:
        
        # Check if this data was succesfully pickled
        if geo_accession in completed:
            continue
            
        # If new data, download data and add to df
        print("********* |", geo_accession, "| ***********")
        logfile.write(geo_accession + ",start\n")
        description, metadata, count_matrix = loader.load_dataset(geo_accession)
        # print("Description: ", description)
        
        # Merge in to existing count matrix DF
        if count_matrix_df.empty:
            count_matrix_df = count_matrix
        else:
            count_matrix_df = pd.merge(count_matrix_df, count_matrix,  how='left', on=["gene", "gene_symbol"], suffixes=[None,"y"])
        
        logfile.write(geo_accession + ",write\n")
        count_matrix_df.to_pickle("download/grein_count_matrix.pkl")
        logfile.write(geo_accession + ",end\n")
        
        break

In [ ]:
# Timimg the loading of data sets
start = time.time()
print(loader.load_overview(10))
end = time.time()
print(end-start)

## Save the data loaded above

In [ ]:
import random
from sklearn.model_selection import train_test_split
import json
import sys

**Count_matrix**

In [ ]:
# still relevant??
# path = os.path.join(os.getcwd(), 'download', 'count_matrix.tsv')
# with open(path, 'w') as file:
#     count_matrix_df.to_csv(path, sep="\t") 

**Description**

In [ ]:
# still relevant??
# Save description. Currently saves description for only one data set at a time
# path = os.path.join('download', 'description.json')
# with open(path, 'w') as file:
#     # Serialize and write the variable to the file
#     json.dump(description, file)

**Metadata**

In [ ]:
# still relevant??

# # Create a dataframe from metadata
# metadata_df = pd.DataFrame.from_dict(metadata)

# # list of row names in metadata_df
# row_names = list(metadata_df.index)

# # Creating a list of rows that we do not require to process this data set
# unneeded_rows = ['Consent', ' ', 'channel_count', 'organism_ch1', 'relation', 'status']
# for index in row_names:
#     if ('contact' in index) or ('date' in index) or ('data_processing' in index) or ('Hash' in index) or ('Date' in index) or ('library' in index):
#         unneeded_rows.append(index)
        
# # remove unnecessary roles from count_matrix
# metadata_df.drop(unneeded_rows, axis = 0)